In [26]:
from utils import *
import numpy as np
import unidecode

In [2]:
db, engine, meta = db_connect()

In [4]:
tables = db_tables(meta)

In [120]:
leg_map = get_leg_map(engine)

In [15]:
list(leg_map.values())[1000]["name"]

{'first': 'Joseph', 'middle': 'David', 'last': 'Waggonner', 'suffix': 'Jr.'}

In [110]:
house_assignments = get_csv("data/house_assignments_103-115.csv")

In [111]:
senate_assignments = get_csv("data/senate_assignments_103-115.csv")

In [115]:
for assignment in house_assignments + senate_assignments:
    assignment["congress"] = assignment.pop("\ufeffcongress")

In [93]:
def find_by_name(leg_map, last_name, first_name):
    def clean_name(s):
        return unidecode.unidecode(s.strip().lower().replace(" ", "").replace("-", ""))
    last_name = clean_name(last_name)
    first_name = clean_name(first_name)
    cur_filter = [leg for leg in leg_map.values() if clean_name(leg["name"]["last"]) == last_name]
    if len(cur_filter) == 1:
        return cur_filter[0]
    if len(cur_filter) == 0:
        cur_filter = [leg for leg in leg_map.values() if clean_name(leg["name"]["last"]) in last_name or
                            last_name in clean_name(leg["name"]["last"])]
    if len(cur_filter) > 1:
        first_filter_strict = [leg for leg in cur_filter if first_name == clean_name(leg["name"]["first"])]
        if len(first_filter_strict) > 0:
            cur_filter = first_filter_strict
            
    if len(cur_filter) == 1:
        return cur_filter[0]
    else:
        # return cur_filter
        print(len(cur_filter))
        raise Exception("Unable to find {} {}".format(first_name, last_name))

In [94]:
for assignment in house_assignments + senate_assignments:
    try: 
        if not isinstance(assignment["bioguide"], str) or len(assignment["bioguide"].strip()) == 0:
                names = assignment["Name"].split(", ")
                last_name = names[0]
                first_name = names[1]
                if(first_name[-1] == "."):
                    first_name = first_name[:-2]
                assignment["bioguide"] = find_by_name(leg_map, last_name, first_name)["id"]["bioguide"]
    except:
        print(assignment["Name"])
        # print(assignment)
        # raise Exception()

[Vacant]
[Vacant]

Majority leader designee


In [46]:
def add_bioguide(assignments, name, bioguide):
    for assignment in assignments:
        if assignment["Name"] == name:
            assignment["bioguide"] = bioguide

In [91]:
find_by_name(leg_map, "", "Jenniffer")

{'id': {'bioguide': 'G000582',
  'govtrack': 412723,
  'wikipedia': 'Jenniffer González',
  'wikidata': 'Q16215793',
  'fec': ['H6PR00082'],
  'opensecrets': 'N00037615',
  'maplight': 2235,
  'google_entity_id': 'kg:/m/027s3vf',
  'cspan': 67353},
 'name': {'first': 'Jenniffer',
  'last': 'González-Colón',
  'official_full': 'Jenniffer González-Colón'},
 'bio': {'gender': 'F', 'birthday': '1976-08-05'},
 'terms': [{'type': 'rep',
   'start': '2017-01-03',
   'end': '2021-01-03',
   'state': 'PR',
   'district': 0,
   'party': 'Republican',
   'phone': '202-225-2615',
   'address': '1609 Longworth House Office Building; Washington DC 20515-5401',
   'office': '1609 Longworth House Office Building',
   'url': 'https://gonzalez-colon.house.gov'}],
 'party': 'Republican',
 'state': 'PR',
 'sponsored_bills': [{'id': 'hr798-115',
   'sponsor_id': 'G000582',
   'type': 'hr',
   'congress': 115},
  {'id': 'hr6209-115', 'sponsor_id': 'G000582', 'type': 'hr', 'congress': 115},
  {'id': 'hr797-1

In [92]:
add_bioguide(senate_assignments + house_assignments, "González, Jenniffer", 'G000582')

In [54]:
list(range(10))[:-2]

[0, 1, 2, 3, 4, 5, 6, 7]

In [81]:
leg_map["R000600"]

{'id': {'bioguide': 'R000600',
  'fec': ['H4AS00036'],
  'govtrack': 412664,
  'opensecrets': 'N00007635',
  'thomas': '02222',
  'house_history': 15032412349,
  'maplight': 2120,
  'wikidata': 'Q18684027',
  'wikipedia': 'Amata Coleman Radewagen',
  'google_entity_id': 'kg:/m/012964p3',
  'cspan': 5188,
  'votesmart': 128380},
 'name': {'first': 'Aumua',
  'last': 'Amata',
  'official_full': 'Aumua Amata Coleman Radewagen'},
 'bio': {'gender': 'F', 'birthday': '1947-12-29'},
 'terms': [{'type': 'rep',
   'start': '2015-01-06',
   'end': '2017-01-03',
   'state': 'AS',
   'district': 0,
   'party': 'Republican',
   'address': '1339 Longworth HOB; Washington DC 20515-5200',
   'office': '1339 Longworth House Office Building',
   'phone': '202-225-8577',
   'url': 'https://radewagen.house.gov',
   'fax': '202-225-8757'},
  {'type': 'rep',
   'start': '2017-01-03',
   'end': '2019-01-03',
   'state': 'AS',
   'district': 0,
   'party': 'Republican',
   'phone': '202-225-8577',
   'url': '

In [95]:
save_csv(house_assignments, "data/house_assignments_103-115.csv")

In [96]:
save_csv(senate_assignments, "data/senate_assignments_103-115.csv")

In [121]:
for assignment in house_assignments + senate_assignments:
    if len(assignment["bioguide"]) == 0:
        continue
    leg = leg_map[assignment["bioguide"]]
    if "committees" not in leg:
        leg["committees"] = {}
    committees = leg["committees"]
    if "congress" not in assignment:
        raise Exception(assignment)
    congress = assignment["congress"]
    if congress not in committees:
        committees[congress] = []
    assignment["committee_name"] = assignment["committee_name"].strip().lower()
    committees[congress].append(assignment)

In [122]:
list(leg_map.values())[0]["committees"]

{'103': [OrderedDict([('name', 'Feinstein, Dianne'),
               ('majority_party', '1'),
               ('rank_in_party', '16'),
               ('party_code', '100'),
               ('appointment_dat', '1/7/93'),
               ('termination_date', '1/3/95'),
               ('senior_party_member', '0'),
               ('committee_seniority', '1'),
               ('period_of_service', '3'),
               ('end_status', '1'),
               ('continuity', '2'),
               ('committee_name', 'appropriations'),
               ('bioguide', 'F000062'),
               ('congress', '103')]),
  OrderedDict([('name', 'Feinstein, Dianne'),
               ('majority_party', '1'),
               ('rank_in_party', '9'),
               ('party_code', '100'),
               ('appointment_dat', '1/7/93'),
               ('termination_date', '1/3/95'),
               ('senior_party_member', '0'),
               ('committee_seniority', '1'),
               ('period_of_service', '2'),
           

In [123]:
len(leg_map)

2438

In [125]:
update_all(engine, tables["leg"], leg_map, verbose = True, increment = 100)

Finished 100 of 2438
Finished 200 of 2438
Finished 300 of 2438
Finished 400 of 2438
Finished 500 of 2438
Finished 600 of 2438
Finished 700 of 2438
Finished 800 of 2438
Finished 900 of 2438
Finished 1000 of 2438
Finished 1100 of 2438
Finished 1200 of 2438
Finished 1300 of 2438
Finished 1400 of 2438
Finished 1500 of 2438
Finished 1600 of 2438
Finished 1700 of 2438
Finished 1800 of 2438
Finished 1900 of 2438
Finished 2000 of 2438
Finished 2100 of 2438
Finished 2200 of 2438
Finished 2300 of 2438
Finished 2400 of 2438
